Import Libs

In [1]:
import pandas as pd

# own packages
from preprocess_data import clean_text
from preprocess_data import lemmatize_text
from model_randomF import model_rf_train
from model_MultinomialNB import model_mnb_train

In [2]:
# get csv data
def read_Data(path):

    # Define column names
    column_names = ['label', 'text']
    return pd.read_csv(path, delimiter='\t', encoding='latin-1', header=None, names=column_names)

In [3]:
## read data
data_train = read_Data(r".\data\training_data_lowercase.csv")
data_test = read_Data(r".\data\testing_data_lowercase_nolabels.csv")    

## preview data
#print(data_train.head, "\n")
#print(data_test.head, "\n")
print("Shape train data:\n", data_train.shape)
print("Shape test data:\n", data_test.shape)

# clean data
data_train['cleaned_text'] = data_train['text'].apply(clean_text)
data_test['cleaned_text'] = data_test['text'].apply(clean_text)

print("Cleand text train: \n", data_train["cleaned_text"].head, "\n")
print("Cleand text test: \n", data_test["cleaned_text"].head, "\n")


## lemmatize data
data_train['lemmatized_text'] = data_train['cleaned_text'].apply(lemmatize_text)
data_test['lemmatized_text'] = data_test['cleaned_text'].apply(lemmatize_text)

print("lemmatized_text train: \n", data_train['lemmatized_text'].head, "\n")
print("lemmatized_text test: \n", data_test['lemmatized_text'].head, "\n")




Shape train data:
 (34152, 2)
Shape test data:
 (9984, 2)
Cleand text train: 
 <bound method NDFrame.head of 0        [donald, trump, sends, embarrassing, new, year...
1        [drunk, bragging, trump, staffer, started, rus...
2        [sheriff, david, clarke, becomes, internet, jo...
3        [trump, obsessed, even, obamas, name, coded, w...
4        [pope, francis, called, donald, trump, christm...
                               ...                        
34147    [tears, rain, thais, gather, late, kings, fune...
34148    [pyongyang, university, needs, nonus, teachers...
34149    [philippine, president, duterte, visit, japan,...
34150    [japans, abe, may, election, many, dont, want,...
34151    [demoralized, divided, inside, catalonias, pol...
Name: cleaned_text, Length: 34152, dtype: object> 

Cleand text test: 
 <bound method NDFrame.head of 0       [copycat, muslim, terrorist, arrested, assault...
1       [wow, chicago, protester, caught, camera, admi...
2       [germanys, fdp, 